# [`wordplay` 🎮 💬](https://github.com/saforem2/wordplay): Shakespeare ✍️

We will be using the [Shakespeare dataset](https://github.com/saforem2/wordplay/blob/main/data/shakespeare/readme.md) to train a (~ small) 10M param LLM _from scratch_.

<div>

<div align="center" style="text-align:center;">

<img src="https://github.com/saforem2/wordplay/blob/main/assets/shakespeare.jpeg?raw=true" width="45%" align="center" /><br>

Image generated from [stabilityai/stable-diffusion](https://huggingface.co/spaces/stabilityai/stable-diffusion) on [🤗 Spaces](https://huggingface.co/spaces).<br>

</div>

<details closed><summary>Prompt Details</summary>

<ul>
<li>Prompt:</li>
<t><q>
Shakespeare himself, dressed in full Shakespearean garb,
writing code at a modern workstation with multiple monitors, hacking away profusely,
backlit, high quality for publication
</q></t>

<li>Negative Prompt:</li>
<t><q>
low quality, 3d, photorealistic, ugly
</q></t>
</ul>

</details>

</div>

## Install / Setup

<div class="alert alert-block alert-warning">
<b>Warning!</b><br>  

**IF YOU ARE EXECUTING ON GOOGLE COLAB**:  

You will need to restart your runtime (`Runtime` $\rightarrow\,$ `Restart runtime`)  
_after_ executing the following cell:

</div>

In [1]:
%%bash

python3 -c 'import wordplay; print(wordplay.__file__)' 2> '/dev/null'

if [[ $? -eq 0 ]]; then
    echo "Has wordplay installed. Nothing to do."
else
    echo "Does not have wordplay installed. Installing..."
    git clone 'https://github.com/saforem2/wordplay'
    python3 wordplay/data/shakespeare_char/prepare.py
    python3 wordplay/data/shakespeare/prepare.py
    python3 -m pip install deepspeed
    python3 -m pip install -e wordplay
fi

/content/wordplay/src/wordplay/__init__.py
Has wordplay installed. Nothing to do.


## Post Install

If installed correctly, you should be able to:

```python
>>> import wordplay
>>> wordplay.__file__
'/path/to/wordplay/src/wordplay/__init__.py'
```

In [2]:
%load_ext autoreload
%autoreload 2
import os

os.environ['COLORTERM'] = 'truecolor'
# If running on MacOS:
# os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
# -----------------------------------------------

from enrich import get_logger
log = get_logger(level='INFO')

import wordplay
log.info(wordplay.__file__)

INFO:root:/content/wordplay/src/wordplay/__init__.py


[2024-02-12 19:12:11][INFO][<ipython-input-2-f3b1b5a6e4af>:14] - /content/wordplay/src/wordplay/__init__.py


## Build Trainer

Explicitly, we:

1. `setup_torch(...)`
2. Build `cfg: DictConfig = get_config(...)`
3. Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4. Build `trainer = Trainer(config)`

In [3]:
import os
import numpy as np
from ezpz import setup
from hydra.utils import instantiate
from wordplay.configs import get_config, PROJECT_ROOT
from wordplay.trainer import Trainer

HF_DATASETS_CACHE = PROJECT_ROOT.joinpath('.cache', 'huggingface')
HF_DATASETS_CACHE.mkdir(exist_ok=True, parents=True)

os.environ['HF_DATASETS_CACHE'] = HF_DATASETS_CACHE.as_posix()

BACKEND = 'DDP'

rank = setup(
    framework='pytorch',
    backend=BACKEND,
    seed=1234,
)

cfg = get_config(
    [
        'data=shakespeare',
        'model=shakespeare',
        'optimizer=shakespeare',
        'train=shakespeare',
        f'train.backend={BACKEND}',
        'train.compile=false',
        'train.dtype=float16',
        'train.max_iters=1000',
        'train.log_interval=50',
        'train.eval_interval=500',
    ]
)
config = instantiate(cfg)

[2024-02-12 19:12:22][INFO][configs:72] - Setting HF_DATASETS_CACHE to /content/wordplay/.cache/huggingface/datasets

[2024-02-12 19:12:22][INFO][utils:160] - NumExpr defaulting to 2 threads.

[2024-02-12 19:12:24][INFO][dist:239] - DistInfo={
    "DEVICE": "cuda",
    "DEVICE_ID": "cuda:0",
    "DISTRIBUTED_BACKEND": "nccl",
    "GPUS_PER_NODE": 1,
    "HOSTFILE": "/content/hostfile",
    "HOSTNAME": "22570efe0a64",
    "HOSTS": "['22570efe0a64']",
    "LOCAL_RANK": 0,
    "MACHINE": "22570efe0a64",
    "NGPUS": 1,
    "NODE_ID": 0,
    "NUM_NODES": 1,
    "RANK": 0,
    "SCHEDULER": "LOCAL",
    "WORLD_SIZE_IN_USE": 1,
    "WORLD_SIZE_TOTAL": 1
}

[2024-02-12 19:12:24][INFO][dist:605] - [0/1] Using device='cuda' with backend='DDP' + 'nccl' for distributed training.

[2024-02-12 19:12:24][INFO][dist:290] - [device='cuda'][rank=0/0][local_rank=0/0][node=0/0]

...

[2024-02-12 19:12:24][INFO][configs:308] - Loading train from /content/wordplay/data/shakespeare_char/train.bin

[2024-02-12 19:12:24][INFO][configs:283] - Rescaling GAS -> GAS // WORLD_SIZE = 1 // 1

[2024-02-12 19:12:24][INFO][configs:433] - Tokens per iteration: 16,384

[2024-02-12 19:12:24][INFO][configs:456] - Using self.ptdtype=torch.float16 on self.device_type='cuda'

[2024-02-12 19:12:24][INFO][configs:462] - Initializing a new model from scratch

### Build `Trainer` object

In [4]:
trainer = Trainer(config)

[2024-02-12 19:12:27][INFO][trainer:277] - Initializing a new model from scratch

[2024-02-12 19:12:28][INFO][model:255] - number of parameters: 10.65M

[2024-02-12 19:12:28][INFO][model:445] - num decayed parameter tensors: 26, with 10,740,096 parameters

[2024-02-12 19:12:28][INFO][model:449] - num non-decayed parameter tensors: 13, with 4,992 parameters

[2024-02-12 19:12:28][INFO][model:465] - using fused AdamW: True

[2024-02-12 19:12:28][CRITICAL][trainer:338] - "devid='cuda:0'"

## Prompt (**prior** to training)

In [5]:
query = "What is an LLM?"
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=16,
    display=False
)
log.info(f"['prompt']: '{query}'")
log.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[2024-02-12 19:12:33][INFO][<ipython-input-5-03afa005b390>:9] - ['prompt']: 'What is an LLM?'

[2024-02-12 19:12:33][INFO][<ipython-input-5-03afa005b390>:10] - ['response']:

What is an LLM?CbQ!!FN?bbs C,sa-Ss::SMqJlNbsssfpQMaSSCvEEEaJriC:?aaa?y
MQVXvsKKQvabbbbEEbbKKJjSSpp-CZnlBa'''CCvVIJjQCCisaatCCJJJJQQJAJJCbSVVVVVV-QLSjsQIaaZBbSCCCpJJC:sGFES:a-wY3H'VjG-sfi$JMva?JaaJYv-asgCC:ssfVICaiVVUYYVVY3QQ-bba,,pttywY:HHIKHQ;SjMlgCN;iJqVIHakwYi KEE?iaa

## Train Model

|  name  |       description            |
|:------:|:----------------------------:|
| `step` | Current training step        |
| `loss` | Loss value                   |
| `dt`   | Time per step (in **ms**)    |
| `sps`  | Samples per second           |
| `mtps` | (million) Tokens per sec     |
| `mfu`  | Model Flops utilization[^1]  |
^legend: #tbl-legend

[^1]: in units of A100 `bfloat16` peak FLOPS

In [6]:
trainer.config.device_type

'cuda'

In [7]:
from rich import print

print(trainer.model)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(65, 384)
    (wpe): Embedding(256, 384)
    (drop): Dropout(p=0.2, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=384, out_features=1152, bias=False)
          (c_proj): Linear(in_features=384, out_features=384, bias=False)
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=384, out_features=1536, bias=False)
          (act_fn): GELU(approximate='none')
          (c_proj): Linear(in_features=1536, out_features=384, bias=False)
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=384, out_features=65, bias=False)
)

## (partial) Training:

We'll first train for 500 iterations and then evaluate the models performance on the same prompt:

> What is an LLM?

In [8]:
trainer.train(train_iters=500)

  0%|          | 0/500 [00:00<?, ?it/s]

                              Training Legend                               
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃    abbr    ┃ desc                                                        ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    step    │ Current training iteration                                  │
│    loss    │ Loss value                                                  │
│     dt     │ Elapsed time per training step (measured in **ms**)         │
│    dtf     │ Elapsed time per forward step (measured in **ms**)          │
│    dtb     │ Elapsed time per backward step (measured in **ms**)         │
│    sps     │ Samples per second                                          │
│    mtps    │ Tokens per second, measured in MEGA (1 x 10^6) tokens / sec │
│    mfu     │ Model flops utilization                                     │
│ train_loss │ Training loss value                                         │
│  val_loss  │ Validation loss value                                       │
└────────────┴─────────────────────────────────────────────────────────────┘

 10%|▉         | 49/500 [00:22<01:07,  6.69it/s]

[2024-02-12 19:13:33][INFO][trainer:768] - step=50 loss=2.5077 dt=127.6917 dtf=12.6559 dtb=15.7795 sps=7.8314 mtps=0.1283 mfu=2.9182 train_loss=4.3000 val_loss=4.2911

 20%|█▉        | 99/500 [00:29<00:56,  7.04it/s]

...

 90%|████████▉ | 449/500 [01:18<00:06,  7.49it/s]

[2024-02-12 19:14:29][INFO][trainer:768] - step=450 loss=1.6809 dt=131.0051 dtf=10.7490 dtb=14.2563 sps=7.6333 mtps=0.1251 mfu=2.6921 train_loss=4.3000 val_loss=4.2911

100%|█████████▉| 499/500 [01:25<00:00,  7.41it/s]

[2024-02-12 19:14:36][INFO][trainer:768] - step=500 loss=1.6046 dt=133.9231 dtf=17.5336 dtb=22.3624 sps=7.4670 mtps=0.1223 mfu=2.7011 train_loss=4.3000 val_loss=4.2911

100%|██████████| 500/500 [01:25<00:00,  5.87it/s]


In [10]:
import time

query = "What is an LLM?"
t0 = time.perf_counter()
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=16,
    display=False
)
log.info(f'took: {time.perf_counter() - t0:.4f}s')
log.info(f"['prompt']: '{query}'")
log.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[2024-02-12 19:16:26][INFO][<ipython-input-10-18639f76aec6>:12] - took: 1.8003s

[2024-02-12 19:16:26][INFO][<ipython-input-10-18639f76aec6>:13] - ['prompt']: 'What is an LLM?'

[2024-02-12 19:16:26][INFO][<ipython-input-10-18639f76aec6>:14] - ['response']:

What is an LLM?

SOMEY:
That then to false then?

POLIXENES:
I were are well the provost.

CORIOLANUS:
Now do, my screed far fair to at metwer to begge,
With be a ting of fray tear her discreed,
And to honour not born suchild shame make would wedge as them.

VIRGIM:
Good

## Resume Training...

In [11]:
trainer.train()

  0%|          | 0/1000 [00:00<?, ?it/s]

[2024-02-12 19:17:21][INFO][trainer:666] - Saving checkpoint to: /content

[2024-02-12 19:17:21][INFO][trainer:667] - Saving model to: /content/model.pth

[2024-02-12 19:17:22][INFO][configs:132] - Appending /content to /content/wordplay/src/ckpts/checkpoints.log

  5%|▍         | 49/1000 [00:23<02:17,  6.92it/s]

...

 95%|█████████▍| 949/1000 [02:43<00:06,  7.41it/s]

[2024-02-12 19:19:49][INFO][trainer:768] - step=1450 loss=1.2527 dt=137.0243 dtf=11.0932 dtb=17.9141 sps=7.2980 mtps=0.1196 mfu=2.6906 train_loss=1.2709 val_loss=1.5249

100%|█████████▉| 999/1000 [02:50<00:00,  7.33it/s]

[2024-02-12 19:19:55][INFO][trainer:768] - step=1500 loss=1.2294 dt=134.9553 dtf=13.1609 dtb=15.5829 sps=7.4099 mtps=0.1214 mfu=2.6976 train_loss=1.2709 val_loss=1.5249

100%|██████████| 1000/1000 [02:50<00:00,  5.87it/s]


## Evaluate Model

In [12]:
import time

query = "What is an LLM?"
t0 = time.perf_counter()
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=2,
    display=False
)
log.info(f'took: {time.perf_counter() - t0:.4f}s')
log.info(f"['prompt']: '{query}'")
log.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[2024-02-12 19:20:12][INFO][<ipython-input-12-99b80f46fd99>:12] - took: 1.8187s

[2024-02-12 19:20:12][INFO][<ipython-input-12-99b80f46fd99>:13] - ['prompt']: 'What is an LLM?'

[2024-02-12 19:20:12][INFO][<ipython-input-12-99b80f46fd99>:14] - ['response']:

What is an LLM? what is there?
Then the matter, the man that the would not shame,
They shall not see him to be thus and will be there.

KING RICHARD III:
Then I'll not seek the charge of my son:
But where it is, that I should be so much a man
To be the bastard of the chi